In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

from dotenv import load_dotenv

load_dotenv('/workspaces/ml-learning/src/airtable-chatbot/.env', override=True)

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

### 2024-02-18

In [ ]:
from llama_index.core import ServiceContext, load_index_from_storage, StorageContext
from llama_index.core.indices import VectorStoreIndex, ListIndex
from llama_index.llms.openai import OpenAI

from llama_index.legacy.vector_stores.faiss import FaissVectorStore
from llama_index.legacy.vector_stores import SupabaseVectorStore
from llama_index.core.readers.base import BaseReader
from llama_index.core import SQLDatabase
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.tools import QueryEngineTool, ToolMetadata, RetrieverTool

from modules.reader import CustomAirtableReader
from modules.airtableprompts  import TEXT_TO_SQL_PROMPT

from typing import Any, Dict, Optional, Type, List
import os
from pathlib import Path
import logging
# import faiss

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

SUPABASE_CONNECTION_STRING=os.getenv('SUPABASE_CONNECTION_STRING')
SQLITE_MEMORY_CONNECTION_STRING="sqlite:///:memory:"
DB_CONNECTION=SUPABASE_CONNECTION_STRING

#https://chartio.com/resources/tutorials/how-to-execute-raw-sql-in-sqlalchemy/
from sqlalchemy import (
    insert,
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Boolean,
    Integer,
    select,
    column,
)

In [ ]:
# Import required libraries
from dotenv import load_dotenv, find_dotenv
from itertools import zip_longest
import os
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import streamlit as st
from streamlit_chat import message

from sidebar_st import Sidebar
from modules.chat_agent import ChatAgent
from modules.chathistory import ChatHistory
from modules.reader import CustomAirtableReader
from modules.indexer import Indexer
from modules.airtableconfig import INDEX_NAMES, AIRTABLE_CONFIG

# setup indexer
    
config = AIRTABLE_CONFIG['BuildBountyMembersGenAI']

reader = CustomAirtableReader()

index_name = INDEX_NAMES[config['TABLE']]

### Update content of sql table

In [ ]:
reader = CustomAirtableReader()

In [ ]:
reader.refresh_table_manual_upsert()

In [ ]:
documents = reader._extract_member_documents()

In [ ]:
from modules.reader import CustomAirtableReader
from modules.indexer import Indexer

reader = CustomAirtableReader()



In [ ]:
indexer = Indexer(reader)

indexer._buildVectorStoreIndex_supabase()

In [ ]:
from pprint import pprint
pprint(documents[12])


In [ ]:
reader.find_member_from_build_updates('andrew suryanto')

In [ ]:
rows = reader.find_member_by_name('andrew')

In [ ]:
rows[0][0]

In [ ]:
members = reader.member_data

In [ ]:
members[0]

In [ ]:
members[0]['fields']['Name']

In [ ]:
anthonys = [member for member in members if member['fields'].get('Name', '')=='Anthony Lu']

In [ ]:
anthonys

In [ ]:
build_updates = reader.build_update_data

In [ ]:
build_updates[0]


In [ ]:
names = set([ b['fields']['Full name'] for b in build_updates ])

In [ ]:
names

#### build_update_data

In [ ]:
build_updates = reader.build_update_data

In [ ]:
from modules.helpers import pickleSave
pickleSave(build_updates, 'build_updates', folder=".", silent=False)

In [ ]:
build_updates[0]

#### Extract documents full

In [ ]:
docs = reader.extract_documents_full()


In [ ]:
docs[0].metadata

In [ ]:
emily = [doc for doc in docs if doc.metadata['member_name']=='Emily Price']

### Indexer

In [ ]:
reader.extract_nodes()

In [ ]:
indexer = Indexer(reader)

In [ ]:
retriever = indexer.semantic_retriever

In [ ]:
retriever.query('How many build updates for Andrew?')

In [ ]:
engine = indexer.semantic_query_engine

In [ ]:
engine.query('Emily Price')

In [ ]:
node_dict = {
    'id_': 'rec0GfJPFEi1XdvEk', 
    'embedding': None, 
    'metadata': 
        { 'airtable_id': 'rec0GfJPFEi1XdvEk', 
         'skills': 'Domain expert', 
         'member_name': 'Emily Price', 
         'linkedin_url': 'https://www.linkedin.com/in/emilyclareprice/', 
         'referrer_name': 'N/A', 
         'keen_for_ai_meetup': True, 
         'accepted': False, 
         'record_type': 'build_club_members', 
         'extracted_timestamp': <built-in function time>}, 'excluded_embed_metadata_keys': [], 'excluded_llm_metadata_keys': [], 'relationships': {}, 'text': "MEMBER DETAILS:\n\nnMember name: Emily Price\nLinkedIn Url: https://www.linkedin.com/in/emilyclareprice/\nSkills: ['Domain expert']\nBuild Project: Tallystone is a smarter way to manage your governance-level transactions, decisions and stakeholders. Our corporate actions workspace allows you to create and share verifiable, inalterable and inarguable records in real-time as corporate actions are processed.\nPast work: I haven't built, but I wrote my Law degree honours thesis on whether AI can be used to sentence criminals.\n", 'start_char_idx': None, 'end_char_idx': None, 'text_template': '{metadata_str}\n\n{content}', 'metadata_template': '{key}: {value}', 'metadata_seperator': '\n', 'class_name': 'TextNode'}

## BUILD VECTOR STORE INDEX INTO SUPABASE

In [ ]:
vector_store_index = indexer._buildVectorStoreIndex_supabase()

## Insert records into Supabase

In [ ]:
#reader.refresh_table()

In [ ]:
# rows = reader.extract_rows2()

In [ ]:
indexer = Indexer(reader)

## Create table in supabase

In [ ]:
DB_CONNECTION

### new way

In [ ]:
from modules.reader import AirTableDesigner

designer = AirTableDesigner()

engine = create_engine(DB_CONNECTION)
default_schema = MetaData()
table = designer._design_build_club_members_table(default_schema)
default_schema.create_all(engine)



In [ ]:
# from sqlalchemy import create_engine
# from sqlalchemy.ext.declarative import declarative_base
# from modules.domain import BuildClubMembers

# engine = create_engine(DB_CONNECTION)
# Base = declarative_base()
# Base.metadata.create_all(engine)

### old way

In [ ]:
engine = create_engine(DB_CONNECTION)
default_schema = MetaData()
table = indexer._design_build_club_members_table(default_schema, "build_club_members")
default_schema.create_all(engine)

In [ ]:
command = "select linkedin_url from build_club_members where name = 'Eric Perez'"
print(f'===SQL command 1 : {command}===')

## Download Build Updates

In [ ]:
from pyairtable import Table, Api
import json

# Replace with your API key, base ID, and table name
from modules.airtableconfig import AIRTABLE_CONFIG

config = AIRTABLE_CONFIG['BuildBountyBuildUpdates']

api = Api(config['TOKEN'])
table = api.table(config['BASE'], config['TABLE'])
records = table.all()

# Process the records as needed
for record in records[:1]:
    json.dumps(record, indent=3)


In [ ]:
from pyairtable import Table, Api
import json

# Replace with your API key, base ID, and table name
from modules.airtableconfig import AIRTABLE_CONFIG

config = AIRTABLE_CONFIG['BuildBountyMembersGenAI']

api = Api(config['TOKEN'])
table = api.table(config['BASE'], config['TABLE'])
records = table.all()

# Process the records as needed
json.dumps(records[3], indent=3)

### images

In [ ]:
import requests

def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        response.raise_for_status()

In [ ]:
try:
    image = download_image('https://v5.airtableusercontent.com/v3/u/25/25/1708336800000/NiFEdvQ1RmQUqX5aO_rtVg/dLQJsZXJEr2aZrPwS1g6DgUSTZzfKkQYhc4beZVD4PWUS2THy3DtRrFoavX59tdGozTanVIjVnMzQOlO-_8zdWaoADLhEVfnqVLoM2tvyHT4q42QzCZKAl_7JZp9rWtbKL54QhPGhecfkAtpz0aCqQ/WFN5tmE3Y4aCdKzyTkiFmRC_ea3GwYEPhFlzLNa2m_c')
except Exception as e:
    print('an exception occurred:', e)


In [ ]:
# from PIL import Image
# import requests
# from io import BytesIO

# url = "https://baobab-poseannotation-appfile.s3.amazonaws.com/media/project_5/images/images01/01418849d54b3005.o.1.jpg"
# response = requests.get(url)
# img = Image.open(BytesIO(response.content))
# img.show()


## 2024-02-20  

- show list of users with actionable button

In [ ]:
build_updates = reader.build_update_data

In [ ]:
members = reader.member_data

In [ ]:
build_updates[0]

In [ ]:
[ bu for bu in build_updates if bu['fields']['Full name']=='Ragy Abraham']

In [ ]:
[ bu for bu in members if bu['fields']['Full name']=='Ragy Abraham']

In [ ]:
text="[('Pegah Khaleghi', 'https://www.linkedin.com/in/pegah-khaleghi/'), ('Ryan Taylor', 'https://www.linkedin.com/in/ryanctaylor95'), ('Johnny Han', 'https://www.linkedin.com/in/johnny-han-503444bb/'), ('xuan tong', 'https://www.linkedin.com/in/xuan-tong-3b7211118/'), ('Loke Li Voon', 'https://www.linkedin.com/in/li-voon-loke-1b40b3155/'), ('Tom Fisher', 'https://www.linkedin.com/in/tfisher1/'), ('Ray Zhu', 'https://www.linkedin.com/in/rayzhudev/'), ('Mohammed Abdullah Chowdhury', 'https://www.linkedin.com/in/mohammed-chowdhury-a584b81b4?trk=contact-info'), ('Param Singh', 'https://www.linkedin.com/in/parambir-singh-769736159/'), ('Francisco Galarza', 'https://www.linkedin.com/in/franciscogalarza/'), ('Carlo Pane', 'https://www.linkedin.com/in/carlo-pane-0b1663a6'), ('aishwarya sahu', 'https://www.linkedin.com/in/aishwaryasahu/'), ('Ashfaqul Bari (Ash)', 'https://www.linkedin.com/in/ash-bari-671b611a4/'), ('Anna Zhang', 'https://www.linkedin.com/in/annazhang343/'), ('Ishan Shah', 'www.linkedin.com/in/ishu3101'), ('Philippe Hong', 'https://www.linkedin.com/in/philippehong/'), ('pramod thebe', 'https://www.linkedin.com/in/pramod-t-658676275/'), ('Kevin Zhu', 'https://www.linkedin.com/in/kevinydzhu/'), ('Aaron xie', 'https://www.linkedin.com/in/aaron404-com/'), ('Gauravjeet Singh', 'https://www.linkedin.com/in/gauravjeet-singh-6968541b9/'), ('Katrin Schmid', 'https://www.linkedin.com/in/katrinschmid/'), ('Xuwei Liang', 'https://www.linkedin.com/in/xuweiliang/'), ('Ben Jackson', 'https://www.linkedin.com/in/ben-jackson-549a16294/'), ('Prabhjot Singh Sodhi', 'https://www.linkedin.com/in/prabhjotsodhi/'), ('Zubin Pratap', 'https://www.linkedin.com/in/zubinpratap/')"

## Vecs create index on collection

In [ ]:
import vecs

# create vector store client
vx = vecs.Client(DB_CONNECTION)

In [ ]:
docs = vx.get_or_create_collection(name='members_and_build_updates', dimension=1536)

In [ ]:
docs.create_index()  #default to HNSW with cosine measure